In [1]:
import pandas as pd
import networkx as nx
import matplotlib
import scipy

In [2]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [17, 8]
%matplotlib notebook

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [14]:
dataset_unidades = pd.read_json("../dataset/departments.json")

# Verify idUnidade uniqueness

In [5]:
unidade_ids = dataset_unidades["idUnidade"].unique()
print("idUnidade is unique: {0}".format(len(unidade_ids) == len(dataset_unidades["idUnidade"])))

idUnidade is unique: True


# Verify whether graph is connected

In [6]:
parents = list()
for i in dataset_unidades["hierarquiaUnidade"]:
    parents.append(i.split(".")[1])

In [7]:
unique_parents_ids = pd.Series(parents).unique()

In [8]:
"""
    For each department, check if its ancestor exists.
"""
nOrphans = 0
for value in list(unique_parents_ids):
    if int(value) not in list(unidade_ids):
        nOrphans += 1
print("Grafico possui {0} orfãos".format(nOrphans))

Grafico possui 0 orfãos


# Get edges

In [9]:
"""
    For each hierarchy string apply: ("x.y.z.w") -> [(x,y), (y,z), (z,w)] 
"""
edges = list()
for hierarquia in dataset_unidades["hierarquiaUnidade"]:
    nos = hierarquia.split(".")[1:-1]
    for index_no in range(len(nos)):
        try:
            edge = (int(nos[index_no]), int(nos[index_no+1]))
            if edge not in edges:
                edges.append(edge)
        except:
            pass

# Graph


In [10]:
G = nx.DiGraph()
"""
    Get node metadada from datasource columns.
    
    Build DiGraph from nodes and edges
"""
for unidade in dataset_unidades.iterrows():
    idUnidade = unidade[1]["idUnidade"]
    attrs = dict()
    for key in unidade[1].index:
        attrs[key] = unidade[1][key]
    G.add_node(idUnidade, **attrs)
G.add_edges_from(edges)
G.remove_edge(605,605)

In [11]:
dag = nx.algorithms.dag

In [12]:
print("G is Directed Acyclic: {0}".format(dag.is_directed_acyclic_graph(G)))

G is Directed Acyclic: True


In [13]:
nx.write_graphml(G, "../output/departments.graphml")